In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight


img_size = (224, 224)
size_batch = 32
epoch_number = 30

train_df = pd.read_csv('train.csv')
train_df['path'] = train_df['image_names'].apply(lambda x: os.path.join('images', x))


train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

train_df['emergency_or_not'] = train_df['emergency_or_not'].astype(str)
val_df['emergency_or_not'] = val_df['emergency_or_not'].astype(str)


datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    shear_range=0.2,
    rescale=1.0 / 255.0
)


train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='emergency_or_not',
    target_size=IMG_SIZE,  # Resizing images to 224x224
    batch_size=size_batch,
    class_mode='binary'
)


val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='path',
    y_col='emergency_or_not',
    target_size=IMG_SIZE,
    batch_size=size_batch,
    class_mode='binary'
)

class_weights = compute_class_weight('balanced', classes=np.unique(train_df['emergency_or_not'].astype(int)), y=train_df['emergency_or_not'].astype(int))
class_weights = dict(enumerate(class_weights))

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Fine-tune the last 30 layers
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epoch_number,
    class_weight=class_weights,
    callbacks=[lr_scheduler, early_stopping]
)

val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.2f}")


test_df = pd.read_csv('test.csv')
test_df['path'] = test_df['image_names'].apply(lambda x: os.path.join('images', x))


test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='path',
    y_col=None,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode=None,
    shuffle=False
)


predictions = (model.predict(test_generator) > 0.5).astype(int).flatten()
test_df['emergency_or_not'] = predictions

test_df[['image_names', 'emergency_or_not']].to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'")
